# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of native and semantic functions that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

From our own testing, planner works best with more powerful models like `gpt4` but sometimes you might get working plans with cheaper models like `gpt-35-turbo`. We encourage you to implement your own versions of the planner and use different models that fit your user needs.

Read more about planner [here](https://aka.ms/sk/concepts/planner)


In [3]:
!python -m pip install semantic-kernel==0.4.6.dev0


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    AzureChatCompletion,
)

kernel = sk.Kernel()

useAzureOpenAI = True

# Configure AI backend used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service(
        "chat_completion",
        AzureChatCompletion(deployment_name=deployment, endpoint=endpoint, api_key=api_key),
    )
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service(
        "gpt-4",
        OpenAIChatCompletion(ai_model_id="gpt-3.5-turbo-1106", api_key=api_key, org_id=org_id),
    )

Python-dotenv could not parse statement starting at line 20


## It all begins with an ask


In [5]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""

### Providing plugins to the planner

The planner needs to know what plugins are available to it. Here we'll give it access to the `SummarizePlugin` and `WriterPlugin` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset.

You can also include native functions as well. Here we'll add the TextPlugin.


In [6]:
from semantic_kernel.core_plugins.text_plugin import TextPlugin

plugins_directory = "../../samples/plugins/"
summarize_plugin = kernel.import_semantic_plugin_from_directory(plugins_directory, "SummarizePlugin")
writer_plugin = kernel.import_semantic_plugin_from_directory(plugins_directory, "WriterPlugin")
text_plugin = kernel.import_plugin(TextPlugin(), "TextPlugin")

Define your ASK. What do you want the Kernel to do?


# Basic Planner


Let's start by taking a look at a basic planner. The `BasicPlanner` produces a JSON-based plan that aims to solve the provided ask sequentially and evaluated in order.


In [7]:
from semantic_kernel.planning.basic_planner import BasicPlanner

planner = BasicPlanner()

In [8]:
basic_plan = await planner.create_plan(ask, kernel)

AttributeError: 'BasicPlanner' object has no attribute 'create_plan'

In [ ]:
print(basic_plan.generated_plan)

You can see that the Planner took my ask and converted it into an JSON-based plan detailing how the AI would go about solving this task, making use of the plugins that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call in order to fulfill the user ask. The output of each step of the plan becomes the input to the next function.


Let's also define an inline plugin and have it be available to the Planner. Be sure to give it a function name and plugin name.


In [ ]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(
    prompt_template=sk_prompt,
    function_name="shakespeare",
    plugin_name="ShakespearePlugin",
    max_tokens=2000,
    temperature=0.8,
)

Let's update our ask using this new plugin


In [ ]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

new_plan = await planner.create_plan(ask, kernel)

In [ ]:
print(new_plan.generated_plan)

### Executing the plan


Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.


In [ ]:
results = await planner.execute_plan(new_plan, kernel)

In [ ]:
print(results)

# The Plan Object Model


To build more advanced planners, we need to introduce a proper Plan object that can contain all the necessary state and information needed for high quality plans.

To see what that object model is, look at (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/plan.py)


# Sequential Planner


The sequential planner is an XML-based step-by-step planner. You can see the prompt used for it here (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/sequential_planner/Plugins/SequentialPlanning/skprompt.txt)


In [ ]:
from semantic_kernel.planning import SequentialPlanner

planner = SequentialPlanner(kernel)

In [ ]:
sequential_plan = await planner.create_plan(goal=ask)

To see the steps that the Sequential Planner will take, we can iterate over them and print their descriptions


In [ ]:
for step in sequential_plan._steps:
    print(step.description, ":", step._state.__dict__)

Let's ask the sequential planner to execute the plan.


In [ ]:
result = await sequential_plan.invoke_async()

In [ ]:
print(result)

# Action Planner


The action planner takes in a list of functions and the goal, and outputs a **single** function to use that is appropriate to meet that goal.


In [ ]:
from semantic_kernel.planning import ActionPlanner

planner = ActionPlanner(kernel)

Let's add more plugins to the kernel


In [ ]:
from semantic_kernel.core_plugins import (
    FileIOPlugin,
    MathPlugin,
    TextPlugin,
    TimePlugin,
)

kernel.import_plugin(MathPlugin(), "math")
kernel.import_plugin(FileIOPlugin(), "fileIO")
kernel.import_plugin(TimePlugin(), "time")
kernel.import_plugin(TextPlugin(), "text")

In [ ]:
ask = "What is the sum of 110 and 990?"

In [ ]:
plan = await planner.create_plan(goal=ask)

In [ ]:
result = await plan.invoke_async()

In [ ]:
print(result)

# Stepwise Planner


Stepwise Planner is based off the paper from MRKL (Modular Reasoning, Knowledge and Language) and is similar to other papers like ReACT (Reasoning and Acting in Language Models). At the core, the stepwise planner allows for the AI to form "thoughts" and "observations" and execute actions based off those to achieve a user's goal. This continues until all required functions are complete and a final output is generated.

See a video walkthrough of Stepwise Planner [here.](https://youtu.be/DG_Ge1v0c4Q?si=T1CHaAm1vV0mWRHu)


In [ ]:
from semantic_kernel.planning import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import (
    StepwisePlannerConfig,
)

Let's create a Bing Search native plugin that we can pass in to the Kernel.

Make sure you have a Bing Search API key in your `.env` file

(https://www.microsoft.com/en-us/bing/apis/bing-web-search-api)


In [ ]:
class WebSearchEnginePlugin:
    """
    A search engine plugin.
    """

    from semantic_kernel.orchestration.kernel_context import KernelContext
    from semantic_kernel.plugin_definition import (
        kernel_function,
        kernel_function_context_parameter,
    )

    def __init__(self, connector) -> None:
        self._connector = connector

    @kernel_function(description="Performs a web search for a given query", name="searchAsync")
    @kernel_function_context_parameter(
        name="query",
        description="The search query",
    )
    async def search(self, query: str, context: KernelContext) -> str:
        query = query or context.variables.get("query")[1]
        result = await self._connector.search(query, num_results=5, offset=0)
        return str(result)

In [ ]:
from semantic_kernel.connectors.search_engine import BingConnector

BING_API_KEY = sk.bing_search_settings_from_dot_env()
connector = BingConnector(BING_API_KEY)
kernel.import_plugin(WebSearchEnginePlugin(connector), plugin_name="WebSearch")

Let's also add a couple more plugins


In [ ]:
from semantic_kernel.core_plugins.math_plugin import MathPlugin
from semantic_kernel.core_plugins.time_plugin import TimePlugin

kernel.import_plugin(TimePlugin(), "time")
kernel.import_plugin(MathPlugin(), "math")

In [ ]:
planner = StepwisePlanner(kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000))

Now let's do a more complicated ask that will require planner to make a call to Bing to get the latest information.


In [ ]:
ask = """How many total championships combined do the top 5 teams in the NBA have?"""

plan = planner.create_plan(goal=ask)

In [ ]:
result = await plan.invoke_async()

In [ ]:
print(result)

Let's see the steps that the AI took to get to the answer.


In [ ]:
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:", step.description)
    print("Function:", step.plugin_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print("  Output:\n", str.replace(result[step._outputs[0]], "\n", "\n  "))